In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

import pickle
import random

In [2]:
random.seed(69)

In [3]:
from fetch_data import DataFetcher

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
table = pd.read_html(url)
df = table[0]
symbol_list = df['Symbol'].tolist()

In [5]:
len(symbol_list), len(set(symbol_list))

(503, 503)

In [6]:
df['Date added'] = pd.to_datetime(df['Date added'])

In [7]:
df.dtypes

Symbol                           object
Security                         object
GICS Sector                      object
GICS Sub-Industry                object
Headquarters Location            object
Date added               datetime64[ns]
CIK                               int64
Founded                          object
dtype: object

In [8]:
datetime.strptime('2019', '%Y')

datetime.datetime(2019, 1, 1, 0, 0)

In [9]:
df_1 = df[df['Date added'] < datetime.strptime('2019', '%Y')]
df_1

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
497,XEL,Xcel Energy,Utilities,Multi-Utilities,"Minneapolis, Minnesota",1957-03-04,72903,1909
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [10]:
df_1_symbol_uni = list(df_1.Symbol.unique())

In [11]:
len(df_1_symbol_uni), len(set(df_1_symbol_uni))

(408, 408)

In [12]:
data_fetcher = DataFetcher(
    api_key = os.environ.get('ALPACA_API_KEY'),
    secret_key = os.environ['ALPACA_SECRET_KEY'],
)

In [13]:
data_fetcher.fetch_data(
    symbol_list = tuple(df_1_symbol_uni),
    years_back = 5)

found null values at : Index(['HWM', 'VTRS'], dtype='object', name='symbol')
number of unique symbols in raw data : 406


In [14]:
data_fetcher.create_datasets()
df_1_raw_data = data_fetcher.get_dataset_()

In [15]:
df_1_raw_data.symbol.nunique()

406

In [16]:
len(list(df_1_raw_data.symbol.unique())), len(set(list(df_1_raw_data.symbol.unique())))

(406, 406)

In [17]:
mask = (df_1_raw_data.symbol.value_counts() == df_1_raw_data.symbol.value_counts().max())
df_1_raw_data = df_1_raw_data[df_1_raw_data.symbol.isin(mask[mask].index)]
df_1_raw_data.head()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap,target_,return_
0,AAPL,2019-10-22,58.26,58.51,57.89,57.97,95050404.0,180734.0,58.24,64.52,0.112989
1,AAPL,2019-10-23,58.49,58.76,58.28,58.75,84632820.0,156383.0,58.58,63.77,0.085447
2,AAPL,2019-10-24,59.07,59.14,58.42,58.85,77050184.0,144677.0,58.73,63.49,0.078845
3,AAPL,2019-10-25,58.74,59.61,58.68,59.57,78842208.0,142213.0,59.25,63.43,0.064798
4,AAPL,2019-10-28,59.77,60.22,59.60,60.17,102744936.0,190483.0,60.00,64.54,0.072628


In [18]:
len(mask[~mask].index)

0

In [19]:
snp_unique_100_2019_symb = random.sample(list(df_1_raw_data.symbol.unique()), k = 100)
with open("/home/naradaw/dev/Charles_Schwab/data/symbol_universe/snp_unique_100_2019", "wb") as fp:
    pickle.dump(snp_unique_100_2019_symb, fp)

In [21]:

with open("/home/naradaw/dev/Charles_Schwab/data/symbol_universe/snp_unique_100_2019", "rb") as fp:
    test_uni = pickle.load(fp)

In [22]:
len(test_uni), len(set(test_uni))

(100, 100)